In [1]:
from tqdm import tqdm

for i in tqdm(range(10000)):
    pass

100%|██████████| 10000/10000 [00:00<00:00, 5895844.81it/s]


In [2]:
from marubatsu import Marubatsu
from collections import defaultdict

def ai_match(ai, params=[{}, {}], match_num=10000):
    print(f"{ai[0].__name__} VS {ai[1].__name__}")
    
    mb = Marubatsu()

    # ai[0] VS ai[1] と ai[1] VS a[0] の対戦を match_num 回行い、通算成績を数える
    count_list = [ defaultdict(int), defaultdict(int)]
    for _ in tqdm(range(match_num)):
        count_list[0][mb.play(ai, params=params, verbose=False)] += 1
        count_list[1][mb.play(ai=ai[::-1], params=params[::-1], verbose=False)] += 1

    # ai[0] から見た通算成績を計算する
    count_list_ai0 = [
        # ai[0] VS ai[1] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[0][Marubatsu.CIRCLE],
            "lose": count_list[0][Marubatsu.CROSS],
            "draw": count_list[0][Marubatsu.DRAW],
        },
        # ai[1] VS ai[0] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[1][Marubatsu.CROSS],
            "lose": count_list[1][Marubatsu.CIRCLE],
            "draw": count_list[1][Marubatsu.DRAW],
        },
    ]           

    # 両方の対戦の通算成績の合計を計算する
    count_list_ai0.append({})
    for key in count_list_ai0[0]:
        count_list_ai0[2][key] = count_list_ai0[0][key] + count_list_ai0[1][key]

    # それぞれの比率を計算し、ratio_list に代入する
    ratio_list = [ {}, {}, {} ]
    for i in range(3):
        for key in count_list_ai0[i]:
            ratio_list[i][key] = count_list_ai0[i][key] / sum(count_list_ai0[i].values())
            
    # 各行の先頭に表示する文字列のリスト
    item_text_list = [ Marubatsu.CIRCLE, Marubatsu.CROSS, "total" ]    
    
    # 通算成績の回数と比率の表示
    width = max(len(str(match_num * 2)), 7)
    diff_list = [ ("count", count_list_ai0, f"{width}d"),
                  ("ratio", ratio_list, f"{width}.1%") ]
    for title, data, format in diff_list:
        print(title, end="")
        for key in data[0]:
            print(f" {key:>{width}}", end="")
        print()
        for i in range(3):
            print(f"{item_text_list[i]:5}", end="")
            for value in data[i].values():
                print(f" {value:{format}}", end="")
            print()
        print()

In [3]:
from tree import Mbtree
from ai import ai_gt1, ai2s

mbtree = Mbtree.load("dfscore")
ai_match(ai=[ai_gt1, ai2s], params=[{"mbtree":mbtree}, {}])

ai_gt1 VS ai2s


100%|██████████| 10000/10000 [00:31<00:00, 315.49it/s]

count     win    lose    draw
o        9697       0     303
x        7786       0    2214
total   17483       0    2517

ratio     win    lose    draw
o       97.0%    0.0%    3.0%
x       77.9%    0.0%   22.1%
total   87.4%    0.0%   12.6%



In [4]:
def calc_bestmoves(self, node):
    bestmoves = []
    for move, childnode in node.children_by_move.items():
        if node.score == childnode.score:
            bestmoves.append(move)
    node.bestmoves = bestmoves
    
    for childnode in node.children:
        self.calc_bestmoves(childnode)
        
Mbtree.calc_bestmoves = calc_bestmoves       


In [5]:
mbtree.calc_bestmoves(mbtree.root)

In [6]:
print(mbtree.root.bestmoves)

[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]


In [7]:
from random import choice

def ai_gt2(mb, mbtree):
    node = mbtree.root
    for move in mb.records[1:]:
        node = node.children_by_move[move]

    return choice(node.bestmoves)

In [8]:
ai_match(ai=[ai_gt2, ai2s], params=[{"mbtree":mbtree}, {}])

ai_gt2 VS ai2s


100%|██████████| 10000/10000 [00:32<00:00, 304.67it/s]

count     win    lose    draw
o        9673       0     327
x        7711       0    2289
total   17384       0    2616

ratio     win    lose    draw
o       96.7%    0.0%    3.3%
x       77.1%    0.0%   22.9%
total   86.9%    0.0%   13.1%



In [9]:
mb = Marubatsu()
mb2 = Marubatsu()

In [10]:
nodelist_by_mb = { mb: mbtree.root }
print(nodelist_by_mb[mb2])

KeyError: <marubatsu.Marubatsu object at 0x000001A5A4E0D590>

In [11]:
nodelist_by_mb = {mb.records: mbtree.root}

TypeError: unhashable type: 'list'

In [13]:
a = [1, 2, 3]
b = tuple(a)
print(a)
print(b)

[1, 2, 3]
(1, 2, 3)


In [12]:
a = [1, [2, 3], (4, 5)]
b = tuple(a)
print(a)
print(b)

[1, [2, 3], (4, 5)]
(1, [2, 3], (4, 5))


In [14]:
a = (1, (2, 3))
b = list(a)
print(a)
print(b)

(1, (2, 3))
[1, (2, 3)]


In [15]:
def calc_bestmoves(self, node):
    self.nodelist_by_mb[tuple(node.mb.records)] = node

    bestmoves = []
    for move, childnode in node.children_by_move.items():
        if node.score == childnode.score:
            bestmoves.append(move)
    node.bestmoves = bestmoves
    
    for childnode in node.children:
        self.calc_bestmoves(childnode)
        
Mbtree.calc_bestmoves = calc_bestmoves 

In [16]:
mbtree.nodelist_by_mb = {}
mbtree.calc_bestmoves(mbtree.root)

In [36]:
records = tuple(mbtree.root.mb.records)
root = mbtree.nodelist_by_mb[tuple(records)]
print(root is mbtree.root)

True


In [18]:
def ai_gt3(mb, mbtree):
    node = mbtree.nodelist_by_mb[tuple(mb.records)]
    return choice(node.bestmoves)

In [19]:
ai_match(ai=[ai_gt3, ai2s], params=[{"mbtree":mbtree}, {}])

ai_gt3 VS ai2s


100%|██████████| 10000/10000 [00:32<00:00, 309.37it/s]

count     win    lose    draw
o        9692       0     308
x        7800       0    2200
total   17492       0    2508

ratio     win    lose    draw
o       96.9%    0.0%    3.1%
x       78.0%    0.0%   22.0%
total   87.5%    0.0%   12.5%



In [20]:
def ai_gt4(mb, mbtree):
    return choice(mbtree.bestmoves_by_mb[tuple(mb.records)])

In [21]:
def calc_bestmoves(self, node):
    bestmoves = []
    for move, childnode in node.children_by_move.items():
        if node.score == childnode.score:
            bestmoves.append(move)
    node.bestmoves = bestmoves

    key = tuple(node.mb.records)
    self.nodelist_by_mb[key] = node
    self.bestmoves_by_mb[key] = bestmoves
   
    for childnode in node.children:
        self.calc_bestmoves(childnode)
        
Mbtree.calc_bestmoves = calc_bestmoves   

In [22]:
mbtree.nodelist_by_mb = {}
mbtree.bestmoves_by_mb = {}
mbtree.calc_bestmoves(mbtree.root)

In [23]:
ai_match(ai=[ai_gt4, ai2s], params=[{"mbtree":mbtree}, {}])

ai_gt4 VS ai2s


100%|██████████| 10000/10000 [00:32<00:00, 307.99it/s]

count     win    lose    draw
o        9714       0     286
x        7704       0    2296
total   17418       0    2582

ratio     win    lose    draw
o       97.1%    0.0%    2.9%
x       77.0%    0.0%   23.0%
total   87.1%    0.0%   12.9%



In [24]:
def ai_gt5(mb, bestmoves):
    return choice(bestmoves[tuple(mb.records)])

In [25]:
ai_match(ai=[ai_gt5, ai2s], params=[{"bestmoves":mbtree.bestmoves_by_mb}, {}])

ai_gt5 VS ai2s


100%|██████████| 10000/10000 [00:32<00:00, 308.59it/s]

count     win    lose    draw
o        9632       0     368
x        7774       0    2226
total   17406       0    2594

ratio     win    lose    draw
o       96.3%    0.0%    3.7%
x       77.7%    0.0%   22.3%
total   87.0%    0.0%   13.0%



In [26]:
import pickle
import gzip
mbtree.save("aidata")
with gzip.open("bestmoves.dat", "wb") as f:
    pickle.dump(mbtree.bestmoves_by_mb, f)

save completed.


In [34]:
mbtree.load("aidata")

In [35]:
with gzip.open("bestmoves.dat", "rb") as f:
    bestmoves = pickle.load(f)

In [29]:
from tree import Node

def __init__(self, algo="bf"):
    self.algo = algo
    Node.count = 0
    self.root = Node(Marubatsu())
    if self.algo == "bf":  
        self.create_tree_by_bf()
        self.calc_score_by_bf()
    else:
        self.nodelist = [self.root]
        self.nodelist_by_depth = [[] for _ in range(10)]
        self.nodelist_by_depth[0].append(self.root)
        self.nodenum = 0
        self.create_tree_by_df(self.root)
        self.nodelist_by_score = []
        self.calc_score_by_df(self.root)
    self.nodelist_by_mb = {}
    self.bestmoves_by_mb = {}
    self.calc_bestmoves(self.root)

Mbtree.__init__ = __init__

In [30]:
mbtree2 = Mbtree()
ai_match(ai=[ai_gt4, ai2s], params=[{"mbtree":mbtree2}, {}])

     9 depth 1 node created
    72 depth 2 node created
   504 depth 3 node created
  3024 depth 4 node created
 15120 depth 5 node created
 54720 depth 6 node created
148176 depth 7 node created
200448 depth 8 node created
127872 depth 9 node created
     0 depth 10 node created
total node num = 549946
ai_gt4 VS ai2s


100%|██████████| 10000/10000 [00:31<00:00, 315.24it/s]

count     win    lose    draw
o        9683       0     317
x        7691       0    2309
total   17374       0    2626

ratio     win    lose    draw
o       96.8%    0.0%    3.2%
x       76.9%    0.0%   23.1%
total   86.9%    0.0%   13.1%



In [31]:
%%timeit
ai_gt1(choice(mbtree.nodelist).mb, mbtree)

IndexError: Cannot choose from an empty sequence

In [37]:
mblist = []
for node in mbtree.nodelist:
    if node.mb.status == Marubatsu.PLAYING:
        mblist.append(node.mb)

In [38]:
%%timeit
ai_gt1(choice(mblist), mbtree)

6.41 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [39]:
%%timeit
ai_gt2(choice(mblist), mbtree)

5.66 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
%%timeit
ai_gt3(choice(mblist), mbtree)

3.59 µs ± 54.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [47]:
%%timeit
ai_gt4(choice(mblist), mbtree)

2.81 µs ± 27.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [48]:
%%timeit
ai_gt5(choice(mblist), bestmoves)

3.18 µs ± 39 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
